In [3]:
import json
import sqlite3
import os 

FROM_DIR = 'transformed'

conn = sqlite3.Connection('data.db')
cur = conn.cursor()

# create_embeddings = """
#     CREATE TABLE embeddings (
#         id INTEGER PRIMARY KEY,         
#         vector TEXT              
#     );
# """
# cur.execute(create_embeddings)
# conn.commit()

insert_embeddings = """
    INSERT INTO embeddings (id, vector) VALUES (?, ?);
"""

for file_name in os.listdir(FROM_DIR):
    print(file_name)
    with open(os.path.join(FROM_DIR, file_name), 'rt') as file:
        for line in file:
            j = json.loads(line)
            cur.execute(
                insert_embeddings, 
                (
                    j['id'], 
                    json.dumps(j['embedding'])
                )
            )
            conn.commit()

train-00000-of-00041.jsonl
train-00001-of-00041.jsonl


In [ ]:
import json
import sqlite3
import os 
import pandas as pd

FROM_DIR = 'source'

conn = sqlite3.Connection('data.db')
cur = conn.cursor()

# create_articles = """
#     CREATE TABLE articles (
#         id INTEGER PRIMARY KEY,         
#         url TEXT,
#         title TEXT
#     );
# """
# cur.execute(create_articles)
# conn.commit()

insert_articles = """
    INSERT INTO articles (id, url, title) VALUES (?, ?, ?);
"""

i = 0
j = 0
WINDOW = 1000
buffer = []
for file_name in os.listdir(FROM_DIR):
    print(file_name)
    df = pd.read_parquet(os.path.join(FROM_DIR, file_name))
    while j + WINDOW < len(df):
        data = tuple([tuple(x[: 3]) for x in df.iloc[j : j + WINDOW].values])
        cur.executemany(insert_articles, data)
        conn.commit()
        j += WINDOW
    if j < len(df):
        data = tuple([tuple(x[: 3]) for x in df.iloc[j : j + WINDOW].values])
        cur.executemany(insert_articles, data)    
        conn.commit()
    i += 1
    if i == 2:
        break

train-00000-of-00041.parquet
train-00001-of-00041.parquet
